In [1]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="OE3Jg3u3F6BPO27NPQFe")
project = rf.workspace("takoyati").project("cigarette-vape-detection")
version = project.version(14)
dataset = version.download("yolov9")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Cigarette-Vape-Detection-14 in yolov9:: 100%|██████████| 27712/27712 [00:03<00:00, 7119.01it/s]


In [3]:
import numpy as np
import os
import PIL
from PIL import Image
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
import shutil

# 경로 설정
train_dir = '/content/Cigarette-Vape-Detection-14/train'
labels_dir = os.path.join(train_dir, 'labels')
images_dir = os.path.join(train_dir, 'images')
vape_dir = os.path.join(train_dir, 'vape')
cigarette_dir = os.path.join(train_dir, 'cigarette')

# vape, cigarette 폴더 생성
os.makedirs(vape_dir, exist_ok=True)
os.makedirs(cigarette_dir, exist_ok=True)

# 파일 이동
for label_file in os.listdir(labels_dir):
    with open(os.path.join(labels_dir, label_file), 'r') as f:
        label = int(f.read().split()[0])
        image_name = os.path.splitext(label_file)[0]
        image_path = os.path.join(images_dir, image_name + '.jpg')

        if label == 0:
            dst_dir = vape_dir
        else:
            dst_dir = cigarette_dir

        shutil.copy(image_path, os.path.join(dst_dir, image_name + '.jpg'))


In [4]:
# ImageDataGenerator 설정
data_generator = ImageDataGenerator(rescale=1./255)

# train_generator 생성
train_generator = data_generator.flow_from_directory(
    '/content/Cigarette-Vape-Detection-14/train/rr',
    target_size=(224, 224),
    batch_size=4,
    class_mode='categorical',
)


Found 12114 images belonging to 2 classes.


In [7]:
base_model = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 새로운 레이어 추가
x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)
vgg_model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# 이전 레이어들의 학습을 동결
for layer in base_model.layers:
    layer.trainable = False

vgg_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
vgg_model.fit(train_generator, epochs=30, batch_size=16)

80134624/80134624 [==============================] - 0s 0us/step
Epoch 1/30
3029/3029 [==============================] - 118s 39ms/step - loss: 0.2793 - accuracy: 0.9020
Epoch 2/30
3029/3029 [==============================] - 118s 39ms/step - loss: 0.0866 - accuracy: 0.9673
Epoch 3/30
3029/3029 [==============================] - 118s 39ms/step - loss: 0.0548 - accuracy: 0.9793
Epoch 4/30
3029/3029 [==============================] - 118s 39ms/step - loss: 0.0284 - accuracy: 0.9898
Epoch 5/30
3029/3029 [==============================] - 118s 39ms/step - loss: 0.0274 - accuracy: 0.9895
Epoch 6/30
3029/3029 [==============================] - 118s 39ms/step - loss: 0.0193 - accuracy: 0.9938
Epoch 7/30
3029/3029 [==============================] - 118s 39ms/step - loss: 0.0178 - accuracy: 0.9946
Epoch 8/30
3029/3029 [==============================] - 118s 39ms/step - loss: 0.0182 - accuracy: 0.9940
Epoch 9/30
3029/3029 [==============================] - 118s 39ms/step - loss: 0.0131 - accurac